# Classificador de Categorias dos produtos
Nesse notebook foi desenvolvido um classificador para classificar os produtos em 6 categorias:
- Bebê
- Bijuterias e Jóias
- Decoração
- Lembrancinhas
- Papel e Cia
- Outros

Como o intuito é demonstrar o conceito da solução, então não foram utilizadas técnicas de hyperparameter tunning.

O dataset utilizado pode ser encontrado nesse [link](https://elo7-datasets.s3.amazonaws.com/data_scientist_position/elo7_recruitment_dataset.csv), e ele foi baixado na data 28/03/2022. 

In [1]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from catboost import CatBoostClassifier

from utils.utils import tokenize, save_model
from utils.estimators import FilterColumns, FillMissing, CalcSellerFeatures
from utils.metrics import calc_mean_f1

RANDOM_SEED = 0

In [2]:
df = pd.read_csv('../data/elo7_recruitment_dataset.csv')

In [3]:
df.head()

,product_id,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category
0,11394449,8324141,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,171.890000,1200.0,1,4,244,NaN,Decoração
1,15534262,6939286,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos lon...,2018-04-04 20:55:07,77.670000,8.0,1,5,124,NaN,Papel e Cia
2,16153119,9835835,expositor de esmaltes,1,38,Organizador expositor p/ 70 esmaltes,expositor,2018-10-13 20:57:07,73.920006,2709.0,1,1,59,NaN,Outros
3,15877252,8071206,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,118.770004,0.0,1,1,180,1.0,Bebê
4,15917108,7200773,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,191.810000,507.0,1,6,34,NaN,Decoração


## Train test split

O split temporal foi feito pela ordenação da data de criação do produto. Esse ponto é necessário para evitar vazamento de informação, pois algumas variáveis que foram testadas podem ter efeitos diferentes devido ao tempo. Como por exemplo, a categoria vendida pelo mesmo vendedor do produto. Se fosse feito um split aleatório, vendedores que já pararam de cadastrar novos produtos poderiam ter produtos no treino e no teste, assim tornando o problema mais fácil, porém com resultados divergentes de quando ele seria colocado em produção.

O tamanho dos datasets foram escolhidos arbitrariamente, só tomando o cuidado para que nos conjuntos de validação e teste tivessem registros suficientes para as distribuições das categorias não mudarem tanto.

In [4]:
# 70% for train set
train_size = int(df.shape[0]* 0.7)

# Split between val e test sets
test_size = int((df.shape[0] - train_size)/2)

# Sort dataset
df_sorted = df.sort_values(by='creation_date')

df_train = df_sorted.iloc[:train_size]
df_val = df_sorted.iloc[train_size:train_size+test_size]
df_test = df_sorted.iloc[train_size+test_size:]

A coluna "query" não será utilizada nesse momento para classificação, assumindo que o produto precise ser classificado no momento em que ele é cadastrado. A informação de query poderia ser utilizada como corpus, dado que temos termos de busca relacionado com uma categoria, mas as abordagem adotadas foram mais simples.

As colunas "search_page", "position", "view_counts", "order_counts" também não serão utilizados para a classificação pois essa informação não estará disponível no momento do cadastro.

In [5]:
vars = ['seller_id',
        'title',
        'concatenated_tags',
        'price',
        'weight',
        'express_delivery',
        'minimum_quantity']

target = ['category']

In [6]:
X_train, y_train = df_train[vars], df_train[target]
X_val, y_val = df_val[vars], df_val[target]
X_test, y_test = df_test[vars], df_test[target]

In [7]:
print(f' X_train shape: {X_train.shape}, y_train shape:{y_train.shape}')
print(f' X_val shape: {X_val.shape}, y_val shape:{y_val.shape}')
print(f' X_test shape: {X_test.shape}, y_test shape:{y_test.shape}')

 X_train shape: (26954, 7), y_train shape:(26954, 1)
 X_val shape: (5776, 7), y_val shape:(5776, 1)
 X_test shape: (5777, 7), y_test shape:(5777, 1)


Os módulos de dataprep foram encapsulados no pacote 'utils'. Eles contém algumas limpezas de texto para gerar os vetores tf-idf como remoção de acentos, numerais e pontuação, transformação em letras minúsculas, remoção de stopwords, tokenização e aplicação de steammer. Além disso, foram criados alguns módulos de preechimento de missing values com abordagens padrões de input da mediana ou texto vazio e outras classes para auxiliar a utilização dos pipelines.

## Modelagem

#### Abordagem 1
Utilizar apenas as variáveis textuais 'title' e 'concatenated_tags' transformados em vetores tf-idf. Como classificador foi utilizado uma regressão logística com estratégia de one-vs-all para tratar o target multiclasse. Para contornar o desbalanceamento, utilizar o class_weight do próprio modelo. O tamanho dos vetores de tf-idf foram escolhidos arbitráriamente com o tamanho 1000, para otimizar o tempo de treinamento mas ao mesmo tempo capturar bastante informação.

In [8]:
pipeline_1 = Pipeline([
    ('fill_missing', FillMissing()),
    ('features', FeatureUnion([
        ('title_feats', Pipeline([
            ('filter_title', FilterColumns('title')),
            ('vect_title', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf_title', TfidfTransformer())     
        ])),
        ('tags_feats', Pipeline([
            ('filter_tags', FilterColumns('concatenated_tags')),
            ('vect_tags', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf_tags', TfidfTransformer())     
        ]))
    ])),
    ('clf', MultiOutputClassifier(LogisticRegression(class_weight='balanced', max_iter=1000), n_jobs=5))
])

- Fit

In [9]:
pipeline_1.fit(X_train, y_train);

- Predict

In [10]:
pred_train_1 = pipeline_1.predict(X_train)
pred_val_1 = pipeline_1.predict(X_val)

- Avaliação do Modelo

Como métrica de validação do modelo foi utilizada o f1-score, pois ele considera o precision (taxa de acerto quando o modelo faz uma previsão) que está ligado diretamente com a taxa de acerto para as ações que utilizarão o modelo. E o recall(quantos exemplos verdadeiros da população toda o modelo conseguiu "capturar" com as previsões) que acaba evitando que o modelo deixe de lado alguma classe mais difícil de prever e só foque nas outras. É importante ter uma métrica única para comparar os modelos, portanto essa será a média não ponderada do f1-score de cada classe com exceção de 'outros'. Não foi realizada a ponderação por distribuição das classes, considerando igualmente importante cada classe.

In [11]:
f1_train_1 = calc_mean_f1(y_train, pred_train_1)
print(f'Approach 1 train set (f1-score): {f1_train_1}')

Approach 1 train set (f1-score): 0.8822


In [12]:
f1_val_1 = calc_mean_f1(y_val, pred_val_1)
print(f'Approach 1 val set (f1-score): {f1_val_1}')

Approach 1 val set (f1-score): 0.8176


In [13]:
# Store target classes
classes = np.unique(y_train)

In [14]:
print(classification_report(y_val, pred_val_1, target_names=classes))

                    precision    recall  f1-score   support

              Bebê       0.83      0.87      0.85      1071
Bijuterias e Jóias       0.87      0.94      0.90       110
         Decoração       0.87      0.85      0.86      1149
     Lembrancinhas       0.92      0.84      0.88      2898
            Outros       0.55      0.75      0.63       178
       Papel e Cia       0.50      0.74      0.59       370

          accuracy                           0.84      5776
         macro avg       0.76      0.83      0.79      5776
      weighted avg       0.86      0.84      0.84      5776



A classe com menor performance é 'Papel e Cia' e ela não a classe com menos registros (Bijuterias e Jóias). Isso indica que as variáveis utilizadas não tem tanta informação para distinguir essa classe.

In [15]:
# Save results
results = {}
results['abordagem_1'] = [f1_train_1, f1_val_1]

#### Abordagem 2
Agora será incluído as variáveis numéricas de 'price', 'weight', 'express_delivery', 'minimum_quantity' e flags de categorias que o vendedor do produto já vende além das features textuais utilizadas na abordagem 1. Como classificador será utilizado uma random forest com estratégia de one-vs-all para tratar o target multiclasse. Para contornar o desbalanceamento, utilizar o class_weight do próprio modelo também.

Está sendo utilizando a classe 'CalcSellerFeatures' do pacote 'utils' para transformar o 'seller_id' em um one-hot enconding das classes que ele já vende nos produtos do treino, porém essa variável pode ser problemática por utilizar informação do target e atrapalhar no treinamento do modelo. A hipótese é que o boostrap da random forest só utilizará essa informação em alguns estimadores, e o modelo conseguirá aprender com as outras informações também.

In [16]:
num_vars = ['seller_id',
            'price',
            'weight',
            'express_delivery',
            'minimum_quantity']

pipeline_2 = Pipeline([
    ('fill_missing', FillMissing()),
    ('features', FeatureUnion([
        ('title_feats', Pipeline([
            ('filter_title', FilterColumns('title')),
            ('vect_title', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf_title', TfidfTransformer())     
        ])),
        ('tags_feats', Pipeline([
            ('filter_tags', FilterColumns('concatenated_tags')),
            ('vect_tags', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf_tags', TfidfTransformer())     
        ])),
        ('num_feats', Pipeline([
            ('filter_num_features', FilterColumns(num_vars)),
            ('process_seller_cats', CalcSellerFeatures()),
        ])),
    ])),
    ('rf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced',
                                             random_state=RANDOM_SEED), n_jobs=1))
])

- Fit

In [17]:
_ = pipeline_2.fit(X_train, y_train);

- Predict

In [18]:
pred_train_2 = pipeline_2.predict(X_train)
pred_val_2 = pipeline_2.predict(X_val)

In [19]:
f1_train_2 = calc_mean_f1(y_train, pred_train_2)
print(f'Approach 2 train set (f1-score): {f1_train_2}')

Approach 2 train set (f1-score): 1.0


In [20]:
f1_val_2 = calc_mean_f1(y_val, pred_val_2)
print(f'Approach 2 val set (f1-score): {f1_val_2}')

Approach 2 val set (f1-score): 0.8453


In [21]:
print(classification_report(y_val, pred_val_2, target_names=classes))

                    precision    recall  f1-score   support

              Bebê       0.87      0.86      0.87      1071
Bijuterias e Jóias       0.90      0.90      0.90       110
         Decoração       0.87      0.83      0.85      1149
     Lembrancinhas       0.89      0.94      0.91      2898
            Outros       0.75      0.58      0.65       178
       Papel e Cia       0.78      0.63      0.70       370

          accuracy                           0.87      5776
         macro avg       0.84      0.79      0.81      5776
      weighted avg       0.87      0.87      0.87      5776



Nessa abordagem os resultados foram melhores, mas é um pouco preocupante o treino ter atingido 1 de f1-score, mostrando sinais de overfit.

In [22]:
# Save Results
results['abordagem_2'] = [f1_train_2, f1_val_2]

## Abordagem 3

Mesma abordagem que a 2, porém removendo as features de seller_id para ver o que acontece com a performance.

In [23]:
num_vars = ['price',
            'weight',
            'express_delivery',
            'minimum_quantity']

pipeline_3 = Pipeline([
    ('fill_missing', FillMissing()),
    ('features', FeatureUnion([
        ('title_feats', Pipeline([
            ('filter_title', FilterColumns('title')),
            ('vect_title', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf_title', TfidfTransformer())     
        ])),
        ('tags_feats', Pipeline([
            ('filter_tags', FilterColumns('concatenated_tags')),
            ('vect_tags', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf_tags', TfidfTransformer())     
        ])),
         ('num_features', FilterColumns(num_vars)),
         
    ])),
    ('rf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced',
                                             random_state=RANDOM_SEED)))
])

- Fit

In [24]:
_ = pipeline_3.fit(X_train, y_train);

- Predict

In [25]:
pred_train_3 = pipeline_3.predict(X_train)
pred_val_3 = pipeline_3.predict(X_val)

In [26]:
f1_train_3 = calc_mean_f1(y_train, pred_train_3)
print(f'Approach 3 train set (f1-score): {f1_train_3}')

Approach 3 train set (f1-score): 1.0


In [27]:
f1_val_3 = calc_mean_f1(y_val, pred_val_3)
print(f'Approach 3 val set (f1-score): {f1_val_3}')

Approach 3 val set (f1-score): 0.8296


In [28]:
print(classification_report(y_val, pred_val_3, target_names=classes))

                    precision    recall  f1-score   support

              Bebê       0.91      0.80      0.86      1071
Bijuterias e Jóias       0.96      0.92      0.94       110
         Decoração       0.85      0.89      0.87      1149
     Lembrancinhas       0.87      0.96      0.91      2898
            Outros       0.92      0.48      0.63       178
       Papel e Cia       0.73      0.47      0.57       370

          accuracy                           0.87      5776
         macro avg       0.87      0.75      0.80      5776
      weighted avg       0.87      0.87      0.86      5776



Mesmo removendo as features de seller, o a performance do treino ainda está com o mesmo comportamento. Porém a performance da validação caiu 0.0157.

In [29]:
# Save results
results['abordagem_3'] = [f1_train_3, f1_val_3]

#### Abordagem 4
Nessa abordagem foram utilizadas as mesmas variáveis da abordagem 2 alterando classificador para um catboost. Além dele ser um algoritmo de boosting podendo ser mais potente que um de bagging, o catboost tende a não precisar de tanto tuning de hiperparâmetros como o gxboost. Ao invés de utilizar o MultiOutputClassifier, foi utilizado a função de custo adequada para o problema multiclasse.

In [30]:
# Calculate weights vector manually to use catboost class_weight

weights = compute_class_weight(class_weight='balanced',
                               classes=classes,
                               y=y_train.category)

In [31]:
num_vars = ['seller_id',
            'price',
            'weight',
            'express_delivery',
            'minimum_quantity']

pipeline_4 = Pipeline([
    ('fill_missing', FillMissing()),
    ('features', FeatureUnion([
        ('title_feats', Pipeline([
            ('filter_title', FilterColumns('title')),
            ('vect_title', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf_title', TfidfTransformer())     
        ])),
        ('tags_feats', Pipeline([
            ('filter_tags', FilterColumns('concatenated_tags')),
            ('vect_tags', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf_tags', TfidfTransformer())     
        ])),
        ('num_feats', Pipeline([
            ('filter_num_features', FilterColumns(num_vars)),
            ('process_seller_cats', CalcSellerFeatures()),
        ])),
    ])),
    ('catboost', CatBoostClassifier(loss_function='MultiClass', class_weights=weights,
                random_state=RANDOM_SEED, verbose=False))
])

- Fit

In [32]:
pipeline_4.fit(X_train, y_train);

- Predict

In [33]:
pred_train_4 = pipeline_4.predict(X_train)
pred_val_4 = pipeline_4.predict(X_val)

In [34]:
f1_train_4 = calc_mean_f1(y_train, pred_train_4)
print(f'Approach 4 train set (f1-score): {f1_train_4}')

Approach 4 train set (f1-score): 0.9476


In [35]:
f1_val_4 = calc_mean_f1(y_val, pred_val_4)
print(f'Approach 4 val set (f1-score): {f1_val_4}')

Approach 4 val set (f1-score): 0.6089


In [36]:
print(classification_report(y_val, pred_val_4, target_names=classes))

                    precision    recall  f1-score   support

              Bebê       0.87      0.62      0.73      1071
Bijuterias e Jóias       0.14      0.92      0.24       110
         Decoração       0.86      0.54      0.67      1149
     Lembrancinhas       0.83      0.88      0.86      2898
            Outros       0.54      0.38      0.45       178
       Papel e Cia       0.57      0.54      0.55       370

          accuracy                           0.73      5776
         macro avg       0.63      0.65      0.58      5776
      weighted avg       0.81      0.73      0.75      5776



O modelo está apresentando overfit, com uma performance de validação muito ruim.

In [37]:
# Save Results
results['abordagem_4'] = [f1_train_4, f1_val_4]

## Abordagem 5

Mesma abordagem que a anterior, só que removendo as features de id_seller porque existe a suspeita que elas estão fazendo o modelo ter overfit.

In [38]:
num_vars = ['price',
            'weight',
            'express_delivery',
            'minimum_quantity']

pipeline_5 = Pipeline([
    ('fill_missing', FillMissing()),
    ('features', FeatureUnion([
        ('title_feats', Pipeline([
            ('filter_title', FilterColumns('title')),
            ('vect_title', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf_title', TfidfTransformer())     
        ])),
        ('tags_feats', Pipeline([
            ('filter_tags', FilterColumns('concatenated_tags')),
            ('vect_tags', CountVectorizer(tokenizer=tokenize, max_features=1000)),
            ('tfidf_tags', TfidfTransformer())     
        ])),
        ('num_features', FilterColumns(num_vars)),
      
    ])),
    ('catboost', CatBoostClassifier(loss_function='MultiClass', class_weights=weights,
                random_state=RANDOM_SEED, verbose=False))
])

- Fit

In [39]:
pipeline_5.fit(X_train, y_train);

- Predict

In [40]:
pred_train_5 = pipeline_5.predict(X_train)
pred_val_5 = pipeline_5.predict(X_val)

In [41]:
f1_train_5 = calc_mean_f1(y_train, pred_train_5)
print(f'Approach 5 train set (f1-score): {f1_train_5}')

Approach 5 train set (f1-score): 0.8654


In [42]:
f1_val_5 = calc_mean_f1(y_val, pred_val_5)
print(f'Approach 5 val set (f1-score): {f1_val_5}')

Approach 5 val set (f1-score): 0.8187


In [43]:
print(classification_report(y_val, pred_val_5, target_names=classes))

                    precision    recall  f1-score   support

              Bebê       0.84      0.81      0.83      1071
Bijuterias e Jóias       0.87      0.95      0.91       110
         Decoração       0.87      0.87      0.87      1149
     Lembrancinhas       0.92      0.86      0.89      2898
            Outros       0.56      0.79      0.65       178
       Papel e Cia       0.51      0.72      0.60       370

          accuracy                           0.84      5776
         macro avg       0.76      0.83      0.79      5776
      weighted avg       0.86      0.84      0.85      5776



Deu certo a remoção das variáveis de seller, e modelo está com a performance dos conjuntos de treino e validação próximas indicando que está mais ajustado que os outros, favorecendo sua generalização para novos dados.

In [44]:
# Save Results
results['abordagem_5'] = [f1_train_5, f1_val_5]

## Resultados

In [45]:
results_df = pd.DataFrame(results).T
results_df.columns = ['f1_train', 'f1_val']
results_df

,f1_train,f1_val
abordagem_1,0.8822,0.8176
abordagem_2,1.0000,0.8453
abordagem_3,1.0000,0.8296
abordagem_4,0.9476,0.6089
abordagem_5,0.8654,0.8187


O modelo final escolhido foi a abordagem 5. Apesar da abordagem 2 e 3 possuirem uma melhor métrica no dataset de validação, por sua performance ter sido 1 no dataset de treino, indica a possibilidade de um overfit comprometendo a generalização do resultado para aplicações futuras.

In [46]:
# Save classifier model
final_model = {'name': 'product_category_classifier',
               'version':1.0,
               'model': pipeline_5}

save_model(final_model, '../models/classifier.pkl')

- Test set performance

In [47]:
pred_test = pipeline_5.predict(X_test)

calc_mean_f1(y_test, pred_test)

0.8156

In [48]:
print(classification_report(y_test, pred_test, target_names=classes))

                    precision    recall  f1-score   support

              Bebê       0.83      0.79      0.81       833
Bijuterias e Jóias       0.84      0.88      0.86       172
         Decoração       0.81      0.86      0.83      1249
     Lembrancinhas       0.92      0.82      0.86      2688
            Outros       0.46      0.78      0.58       208
       Papel e Cia       0.65      0.78      0.71       627

          accuracy                           0.82      5777
         macro avg       0.75      0.82      0.78      5777
      weighted avg       0.84      0.82      0.82      5777



O resultado geral entre o dataset de validação e teste são bem próximos utilizando o modelo final. Isso indica que o modelo está generalizando bem o seu comportamento. Porém, ouve uma troca de performance entre as classes.

## Conclusão

O modelo final possui uma performance razoável visto que nem foram testados seu espaço de hiperparâmetros. A informação de quais tipos de produtos o vendedor já vende é interessante, porém ela atrapalha o aprendizado das técnicas de machine learning. Talvez ela possa ser utilizada como uma heurística combinada com a previsão do modelo para impulsionar a performance. E é necessário mais variáveis que discriminem a classe 'Papel e Cia' para alavancar a performance geral.

Como trabalhos futuros de hipóteses que não foram testas nesse estudo:
- Realizar o balanceamento das classes utilizando técnicas de subsample e oversample
- Treinar uma rede neural com cada neurônio de output a probabilidade de uma classe, podendo testar diferentes estruturas para tratar as features de texto e utilizando uma camada de softmax para o output das classes
- Utilizar informação de texto da coluna 'query' para enriquecer os encoders de texto das features de 'title' e 'concatenated_tags'.